[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/stfc/janus-core/blob/main/docs/source/tutorials/python/elasticity.ipynb)

# Elasticity

## Set up environment (optional)

These steps are required to run this tutorial with Google Colab. To do so, uncomment and run the cell below.

This will replace pre-installed versions of `numpy` and `torch` in Colab with versions that are known to be compatible with `janus-core`.

It may be possible to skip the steps that uninstall and reinstall `torch`, which will save a considerable amount of time.

These instructions but may work for other systems too, but it is typically preferable to prepare a virtual environment separately before running this notebook if possible.

In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

# ! pip uninstall numpy -y # Uninstall pre-installed numpy

# ! pip uninstall torch torchaudio torchvision transformers -y # Uninstall pre-installed torch
# ! uv pip install torch==2.5.1 # Install pinned version of torch

# ! uv pip install janus-core[mace,visualise] data-tutorials --system # Install janus-core with MACE and WeasWidget, and data-tutorials

# get_ipython().kernel.do_shutdown(restart=True) # Restart kernel to update libraries. This may warn that your session has crashed.

To ensure you have the latest version of `janus-core` installed, compare the output of the following cell to the latest version available at https://pypi.org/project/janus-core/

In [ ]:
from janus_core import __version__

print(__version__)

## Prepare data and modules

In [ ]:
from weas_widget import WeasWidget

from ase.build import bulk, nanotube
from ase.lattice.cubic import Diamond
from ase.io import read

import numpy as np
import matplotlib.pyplot as plt

from janus_core.calculations.elasticity import Elasticity

Use `data_tutorials` to get the data required for this tutorial:

### Generation of samples

In ```janus_core``` we can calculate the elasticity tensor via [pymatgen](https://github.com/materialsproject/pymatgen).

As an example we will do this for three samples: Aluminium, Diamond, and a Carbon-nanotube. Then compare their relative stiffnesses.

Using the ASE we can build each sample using the utility functions in ```ase.build```.

Firstly Aluminium can be generated using ```ase.build.bulk```.

In [ ]:
al = bulk("Al", crystalstructure="fcc")*(3,3,3)

v=WeasWidget()
v.from_ase(al)
v

Next we can build Diamond,

In [ ]:
diamond = Diamond('C')*(2,2,2)

v=WeasWidget()
v.from_ase(diamond)
v

And finally the Carbon-nanotube,

In [ ]:
nt = nanotube(6, 6, length=4)
# Place into a box
nt.cell = [nt.cell[2][2], nt.cell[2][2], nt.cell[2][2]]
nt.pbc = [True, True, True]
v=WeasWidget()
v.from_ase(nt)
v

### Elasticity calculations

```janus-core``` includes the elasticity calculation type for calculating the elasticity tensor. This is done by first creating a set of deformed (strained) structures. The stress tensor tensor ($\sigma$) is then calculated for each of these deformed structures. Finally the elasticity tensor $C^{ijkl}$ is estimated from the relationship between stress and strain. Or mathematically:

$$ \sigma^{ij} = C^{ijkl} E^{kl} $$

To find $C^{ijkl}$ we can use ```janus_core.calculations.elasticity.Elasticity```. First we construct the calculation object and run the calculation using the ```run()``` method. This will first geometry optimize the input structure, and then generate the set of deformed structures from it. Finally stresses are calculated for each deformed structure.

In [ ]:
elasticity_aluminium = Elasticity(
    struct=al.copy(),
    arch="mace_mp",
    device="cpu",
    model="small",
    calc_kwargs={"default_dtype": "float64"}
).run()

The results will be written to ```Al27-elastic_tensor.dat``` (automatically generated from our structures composition). The file includes the components of the $3\times3\times3\times3$ elasticity tensor in (row-major) Voigt reduced form ($6\times6$), which is preceded by various derived values such as the shear and bulk moduli.

In [ ]:
! cat janus_results/Al27-elastic_tensor.dat

The experimental bulk, shear, and Young's moduli are approximately 76 GPa, 26 GPa, and 68 GPa for Aluminium. Our results are in the right ball park.

For elasticity the main options for user control are the magnitudes of the applied shear and normal strains as well as their number.

By default 4 shear and 4 normal strains are applied. Split equally positively and negatively, and along each possible direction. This means $2\times 4 \times 3 = 24$ total stress calculations.

For example we can increase the shear and normal magnitudes, and apply 16 strains of each type as follows:

In [ ]:
calc_elasticity_aluminium = Elasticity(
    struct=al.copy(),
    arch="mace_mp",
    device="cpu",
    model="small",
    calc_kwargs={"default_dtype": "float64"},
    shear_magnitude=0.3,
    normal_magnitude=0.2,
    n_strains=16
)
elasticity_aluminium = calc_elasticity_aluminium.run()

Using Weas we can get a feel for how the applied deformation are impacting our structure by playing through them like any other trajectory.

In this case, since we saved the calculation object we can directly access the ```deformed_structures```

In [ ]:
v=WeasWidget()
v.from_ase(calc_elasticity_aluminium.deformed_structures)
v

Also note that the deformed structure contain the applied strain within their ```info``` attributes

In [ ]:
s = calc_elasticity_aluminium.deformed_structures[-1].info["strain"]
print(f"(Voigt) Strain recorded in structure: {calc_elasticity_aluminium.strains[-1].voigt}")
print(f"Original strain {calc_elasticity_aluminium.strains[-1]}")

In [ ]:
elasticity_diamond = Elasticity(
    struct=diamond.copy(),
    arch="mace_mp",
    device="cpu",
    model="small",
    write_structures=True,
    calc_kwargs={"default_dtype": "float64"}
).run()

Notice for diamond we used the ```write_structures=True``` key word argument, in this case our results directory will include the file ```C64-generated.extxyz``` and ```C64-minimized-structure.extxyz```.

The latter is simply our initial diamond structure, but geometry optimized. The former is again the set of strained structures used to calculate the elasticity. We can read the file and observe the impact of the strains like so

In [ ]:
v=WeasWidget()
diamond_strained = read("janus_results/C64-generated.extxyz", index=":")
v.from_ase(diamond_strained)
v

In [ ]:
elasticity_nt = Elasticity(
    struct=nt.copy(),
    arch="mace_mp",
    device="cpu",
    model="small",
    calc_kwargs={"default_dtype": "float64"}
).run()

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, layout='constrained')

for tensor, material, marker in zip((elasticity_aluminium, elasticity_diamond, elasticity_nt), ("Aluminium", "Diamond", "Carbon-nanotube"), "o^s"):
    ecs = dict()
    ecs["Bulk modulus"] = tensor.k_voigt
    ecs["Shear modulus"] = tensor.g_voigt
    # Note https://github.com/materialsproject/pymatgen/issues/4435
    ecs["Young's modulus"] = tensor.y_mod/1e9
    theta = [i*2.0*np.pi/(len(ecs)) for i in range(len(ecs))]
    ax.scatter(theta, ecs.values(), marker=marker, label=material)
    ax.set_xticks(theta, ecs.keys())

fig.legend(loc='outside upper right')
fig.suptitle("Elastic modulii for our samples")